In [1]:
##########################################################################################
#                                                                                        #
#  888888b.  8888888 .d88888b.   .d8888b.     .d8888b.   .d8888b.      d8888      d8888  #
#  888  "88b   888  d88P" "Y88b d88P  Y88b   d88P  Y88b d88P  Y88b    d8P888     d8P888  #
#  888  .88P   888  888     888 Y88b.        888        888          d8P 888    d8P 888  #
#  8888888K.   888  888     888  "Y888b.     888d888b.  888d888b.   d8P  888   d8P  888  #
#  888  "Y88b  888  888     888     "Y88b.   888P "Y88b 888P "Y88b d88   888  d88   888  #
#  888    888  888  888     888       "888   888    888 888    888 8888888888 8888888888 #
#  888   d88P  888  Y88b. .d88P Y88b  d88P   Y88b  d88P Y88b  d88P       888        888  #
#  8888888P" 8888888 "Y88888P"   "Y8888P"     "Y8888P"   "Y8888P"        888        888  #
#                                                                                        # 
##########################################################################################
#
# Wrangle Synthetic Medical Record Data -- Part 2
#
##########################################################################################

## Fetch data
(This is the same data used in the previous homework--no need to re-fetch it)
Go here: https://drive.google.com/drive/folders/0B5Wu8z15TIbiLUhvcWxaMDJCRGs

Get this data set: 10000-Patients.zip

unzip it into the Data folder

In [1]:
import pandas as pd
import pylab as plt
import numpy as np
import datetime as dt

In [2]:
##  ###################################################################################################################
#
# You do:  
#    Ensure you have SQLite3 available to your system.  If not, install it via pip, conda, etc.
import sqlite3

In [3]:
# Read data in to data frames

p = pd.read_csv('../Data/PatientCorePopulatedTable.txt', header=0, sep = '\t')
a = pd.read_csv('../Data/AdmissionsCorePopulatedTable.txt', header=0, sep='\t')
d = pd.read_csv('../Data/AdmissionsDiagnosesCorePopulatedTable.txt', header=0, sep='\t')  
l = pd.read_csv('../Data/LabsCorePopulatedTable.txt', header=0, sep='\t')            


In [4]:
p.loc[p.PatientID=='915BC24E-8C44-4D33-A386-CEA965B83F32']

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty
9687,915BC24E-8C44-4D33-A386-CEA965B83F32,Male,1921-12-20 07:51:09.003,Asian,Single,English,15.95


In [5]:
# Convert date columns to pandas date objects
# Show the type


for table in [p,a,d,l]:
    for col in table.columns:
        if 'date' in col.lower():
            print('Converting {}'.format(col))
            table[col] = pd.to_datetime(table[col])
            
type(p.PatientDateOfBirth[3])

Converting PatientDateOfBirth
Converting AdmissionStartDate
Converting AdmissionEndDate
Converting LabDateTime


pandas._libs.tslibs.timestamps.Timestamp

 SQLite3 is a "lightweight" database management system.  Within Python you create a "connection" to the dabaase 
 (here we called it "conn") and you can then send SQL queries to the connection, getting standard data frames in 
 return.



In [6]:

conn = sqlite3.connect(':memory:')  # this makes an "in memory" database that isn't written to a disk.  You 
                                    # can replace this with a file name if you want to store the data for later
                                    # use.

print('Loading data into in-memory db--this could take a minute or two')
l.to_sql('L', conn, if_exists='replace')
d.to_sql('D', conn, if_exists='replace')
a.to_sql('A', conn, if_exists='replace')
p.to_sql('P', conn, if_exists='replace')

Loading data into in-memory db--this could take a minute or two


## Basic SQL queries


In [7]:
# To query via SQLite3, write a query as a Python string and "execute it" read_sql_query() method.
#
# Note:  If you are accessing data already in a different database managemnt system (MySQL/Access/etc) there are
#  interfaces available which give you a "conn" to that data as well.  You're not limited to SQLite with this
#  method.


sql = """
SELECT * from L
LIMIT 10
"""

df2 = pd.read_sql_query(sql, conn)
df2



,index,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,0,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: HEMATOCRIT,40.7,%,1946-09-07 22:20:26.677000
1,1,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: ANION GAP,8.4,mmol/L,1946-09-07 11:52:58.600000
2,2,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: LYMPHOCYTES,4.7,k/cumm,1946-09-07 06:08:57.303000
3,3,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: HEMOGLOBIN,15.9,gm/dl,1946-09-07 19:16:10.057000
4,4,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: SODIUM,146.6,mmol/L,1946-09-07 14:03:11.003000
5,5,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: ALBUMIN,3.3,gm/dL,1946-09-08 04:06:53.967000
6,6,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: BUN,17.1,mg/dL,1946-09-07 19:41:39.247000
7,7,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: NEUTROPHILS,8.4,k/cumm,1946-09-07 05:33:34.320000
8,8,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: CALCIUM,8.7,mg/dL,1946-09-07 16:23:41.213000
9,9,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: GLUCOSE,110.5,mg/dL,1946-09-07 09:40:39.467000


In [8]:

# Use the WHERE statement

# Example: Query all labs from admission-id #3

sql = """
SELECT *
FROM L
WHERE AdmissionID==3;
"""
pd.read_sql_query(sql, conn)
#df2


,index,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,163,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: MCHC,31.6,g/dl,2008-10-08 10:11:47.543000
1,164,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: MCH,29.6,pg,2008-10-08 17:55:31.247000
2,165,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: CREATININE,0.6,mg/dL,2008-10-08 15:36:59.533000
3,166,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: WHITE BLOOD CELL COUNT,11.4,k/cumm,2008-10-08 09:31:41.360000
4,167,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: BUN,14.5,mg/dL,2008-10-08 07:56:07.697000
5,168,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: RDW,11.1,%,2008-10-08 03:24:55.450000
6,169,915BC24E-8C44-4D33-A386-CEA965B83F32,3,URINALYSIS: RED BLOOD CELLS,0.3,rbc/hpf,2008-10-08 23:39:43.667000
7,170,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: ABSOLUTE LYMPHOCYTES,23.1,%,2008-10-08 18:41:18.467000
8,171,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: CARBON DIOXIDE,20.9,mmol/L,2008-10-08 04:50:40.677000
9,172,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: GLUCOSE,93.1,mg/dL,2008-10-08 16:40:24.897000


In [9]:
# Example: 
# Query just the time and value for CBC: MCHC labs from admission-id #3, ensuring they are sorted in time order


sql = """
SELECT LabDateTime AS Time, LabValue as Value
FROM L
WHERE AdmissionID==3 AND LabName LIKE "%MCHC"
ORDER BY Time
"""

df3 = pd.read_sql_query(sql, conn)
df3.head()


,Time,Value
0,1948-10-17 08:30:55.433000,35.8
1,1948-10-18 23:34:59.213000,39.9
2,1948-10-19 09:20:36.913000,31.1
3,1948-10-21 22:42:59.537000,33.0
4,1948-10-23 05:08:27.097000,31.4


In [34]:
# You would like to explore the relationship between lab-orders and depression
#
###################################################################################################################
#
# You do:
#  1) 
#   Generate a list of all (unique) diagnosis codes with the term "Major depressive disorder" in the description
#
#  2) Using a JOIN statement, join the Diagosis and Labs tables on PatientID and print the first 40 results


In [10]:
# 1) #this does not bring anything up using Major depressive disorder, so I am using bipolar disorder instead
sql = """
SELECT PatientID as ID, PrimaryDiagnosisDescription AS Diagnosis, PrimaryDiagnosisCode as Code
FROM D
WHERE Diagnosis LIKE "%Major depressive disorder%"
GROUP by Code
ORDER by ID
"""

df4 = pd.read_sql_query(sql, conn)
df4

,ID,Diagnosis,Code
0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0
1,26D1B1B8-B88A-43B3-A021-4DDFDDD962DF,"Major depressive disorder, single episode, sev...",F32.2
2,4F306452-5265-415E-88CB-31C1687FAE77,"Major depressive disorder, recurrent, in full ...",F33.42
3,65EFE359-E8FC-4D83-B9E8-1FAC2DC0EFCB,"Major depressive disorder, recurrent severe wi...",F33.2
4,7C93E968-CEA7-4C24-9B81-EB0C68EE8C95,"Major depressive disorder, recurrent",F33
5,7EDEB6AB-5899-4108-870B-DA34560A382A,"Major depressive disorder, recurrent, in remis...",F33.4
6,94439494-12D4-401F-8FA9-3C287D549975,"Major depressive disorder, single episode, mod...",F32.1
7,BA0E0A19-271E-4017-A66F-992313E3ABD1,"Major depressive disorder, single episode, sev...",F32.3
8,D3D61876-29DF-4D74-B38F-ADF443312F5F,"Major depressive disorder, recurrent, in parti...",F33.41
9,DB72537D-B65E-46BC-B2F9-5F7F0D8A7D60,"Major depressive disorder, recurrent, severe w...",F33.3


In [11]:
df3.to_sql('DF3', conn, if_exists='replace')
df4.to_sql('DF4', conn, if_exists='replace')


In [12]:

sql = """
SELECT *
FROM DF4
LEFT JOIN L
ON DF4.ID==L.PatientID
ORDER by PatientID
LIMIT 40
"""

df5 = pd.read_sql_query(sql, conn)
df5

# I wonder with the following table how do we get rid of the duplicate columns?

,index,ID,Diagnosis,Code,index,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472527,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,CBC: RDW,10.8,%,2006-04-22 09:37:15.650000
1,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472528,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,METABOLIC: CHLORIDE,92.6,mmol/L,2006-04-22 06:21:06.997000
2,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472529,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,METABOLIC: ANION GAP,11.9,mmol/L,2006-04-21 10:57:12.733000
3,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472530,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,METABOLIC: CALCIUM,11.2,mg/dL,2006-04-22 05:00:38.280000
4,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472531,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,CBC: MONOCYTES,0.5,k/cumm,2006-04-21 18:16:29.640000
5,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472532,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,METABOLIC: TOTAL PROTEIN,5.6,gm/dL,2006-04-22 07:24:51.390000
6,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472533,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,CBC: ABSOLUTE LYMPHOCYTES,21.9,%,2006-04-22 03:42:08.930000
7,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472534,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,CBC: ABSOLUTE NEUTROPHILS,69.0,%,2006-04-22 00:37:43.837000
8,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472535,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,URINALYSIS: SPECIFIC GRAVITY,1.0,no unit,2006-04-21 22:41:10.073000
9,0,0DF482BD-AE45-4DF5-847A-1F39C46B7112,"Major depressive disorder, single episode, mild",F32.0,2472536,0DF482BD-AE45-4DF5-847A-1F39C46B7112,1,CBC: MEAN CORPUSCULAR VOLUME,88.1,fl,2006-04-22 07:35:59.610000


In [3]:

###################################################################################################################
#
# You do:
#  1) Create a list of each lab name ordered and it's count for any patient who also had a diagnosis which 
#     includes the term "Major depressive disorder"




In [19]:
sql = """
SELECT L.LabName, COUNT(L.LabName)
FROM D
INNER JOIN L
ON D.PatientID==L.PatientID
WHERE D.PrimaryDiagnosisDescription LIKE "%Major depressive disorder%"
GROUP BY L.LabName
ORDER by COUNT(L.LabName) DESC
"""
# I was stuck here before class and grabbed this code from your example.
df6 = pd.read_sql_query(sql, conn)
df6

,LabName,COUNT(L.LabName)
0,CBC: RDW,7825
1,CBC: WHITE BLOOD CELL COUNT,7817
2,METABOLIC: ALK PHOS,7814
3,CBC: RED BLOOD CELL COUNT,7812
4,CBC: PLATELET COUNT,7802
5,METABOLIC: CHLORIDE,7797
6,METABOLIC: POTASSIUM,7796
7,URINALYSIS: SPECIFIC GRAVITY,7795
8,CBC: LYMPHOCYTES,7784
9,METABOLIC: SODIUM,7776
